# Mass balance model in OGGM

In this notebook, we will take a look at the default mass balance model used in OGGM. It is a very simple temperature index model that only requires temperature and precipitation as climate input variables. Therefore, we’ll start this session by adding the climate data to our glacier directory.

## OGGM setup

First, we start with the package imports and set up the working directory, as always:

In [ ]:
# first we import the packages we need for this session
from oggm import cfg, utils, workflow, tasks 
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cfg.initialize()

# define working directory
path = 'mb_working_dir'
utils.mkdir(path, reset=False)  # if you set reset=True, everything will be deleted and you can start from a fresh state
cfg.PATHS['working_dir'] = path

In the last session, we defined the elevation band flowline for our glacier. This corresponds to level 2 of the [preprocessed directories](https://docs.oggm.org/en/stable/shop.html#pre-processed-directories). So, we don’t need to repeat the tasks from the previous session, we can start directly from level 2:

In [ ]:
# select the glacier of your choice, you can use the Glims viewer from the first session
rgi_ids = ['RGI60-11.00897']  # Hintereisferner

# we load the outline data from the oggm cluster
prepro_base_url_L1 = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/elev_bands/'
gdirs = workflow.init_glacier_directories(rgi_ids,
                                          from_prepro_level=2,  # here we select level 2
                                          prepro_base_url=prepro_base_url_L1,
                                          prepro_border=80,  # could be 10, 80, 160 or 240
                                         )
gdir = gdirs[0]  # for convenience, we define a single variable for our glacier

Let's have a look at the diagnostics:

In [ ]:
gdir.get_diagnostics()

<div class="alert alert-warning">
    <b>Task</b>: What is different from the last session?
</div>

Your answer here:

## Climatic Input data

OGGM supports several different climate datasets by default. You can find an overview of them in the [OGGM documentation](https://docs.oggm.org/en/stable/climate-data.html#climate-data). The dataset to use can be set with `cfg.PARAMS['baseline_climate']`.

In [ ]:
# You can define which climate data to use
cfg.PARAMS['baseline_climate']

<div class="alert alert-warning">
    <b>Task</b>: What is the GSWP3_W5E5 dataset? What is its resolution? What is the main reason for selecting this dataset?
</div>

Your answere here:

Adding one of the default climate datasets to the glacier directory only requires executing a single task:

In [ ]:
# add climate data to gdir
workflow.execute_entity_task(tasks.process_climate_data, gdirs);

<div class="alert alert-warning">
    <b>Task</b>: Have a look in your glacier's working directory. Is their a new file?
</div>

Your answere here:

There is a built-in function to check which climate data was added to the glacier directory:

In [ ]:
gdir.get_climate_info()

<div class="alert alert-warning">
    <b>Task</b>: Do you understand all the variables?
</div>

Your answere here:

We can also open `climate_historical.nc` and plot the variables:

In [ ]:
ds = xr.open_dataset(gdir.get_filepath('climate_historical'))
ds

Let’s take a look at the monthly temperature and calculate the mean annual temperature:

In [ ]:
# plotting the monthly temperature
ax = ds.temp.plot(label='monthly')

# calculate the mean annual temperature and plot it
ds.temp.resample(time='1YS').mean(dim='time').plot(color='k', label='annual mean')

# add legend and grid
plt.legend(); plt.grid('on');

It’s hard to identify any trends in the annual mean because of the large interannual variability of the monthly temperature. Let’s plot the annual mean on its own:

In [ ]:
ds.temp.resample(time='1YS').mean(dim='time').plot(color='k', label='annual mean')
plt.grid('on')

<div class="alert alert-warning">
    <b>Task</b>: Analyze the annual mean temperature. Is there any visible trend?
</div>

Your answere here:

<div class="alert alert-warning">
    <b>Task</b>: Conduct a similar analysis for the standard deviation of daily temperatures (temp_std) and for precipitation (prcp). However, instead of calculating the annual mean for precipitation, compute the annual sum. Do you notice any trends?
</div>

Your answere here:

## Mass balance observations

### Geodetic mass balance

We use a geodetic mass balance observation to calibrate our Mass Balance model, specifically, the average geodetic observation from 2000 to 2020 by [Hugonnet et al. 2021](https://www.nature.com/articles/s41586-021-03436-z).

<div class="alert alert-warning">
    <b>Task</b>: What is a geodetic mass balance? If you can not remember have a look at the slides from the lecture!
</div>

Your answer here:

A nice visualisation of this observation can be found in this [Theia app](http://maps.theia-land.fr/theia-cartographic-layers.html?year=2021&month=09&collection=glaciers).

<div class="alert alert-warning">
    <b>Task</b>: Open the Theia app, select `Glacier Elevation Change`, and activate `compare` (upper right corner). Go to one glacier in the alps and compare the two periods `2000-2009` and `2010-2019`. What do you observe?
</div>

Your answer here:

In the model, we don’t use the distributed information you just explored in the app, as it comes with large uncertainties (which are not shown in the app). Instead, we use a single, glacier-integrated value for the entire period from 2000 to 2020.

In [ ]:
from oggm.utils import get_geodetic_mb_dataframe

get_geodetic_mb_dataframe().loc[gdir.rgi_id]

In the data shown above, you can see that the error of the observation (`err_dmdtda`) is the smallest for the entire 20-year period compared to the two separate 10-year periods.

### Glaciological mass balance

We also have direct observations for a few glaciers.

<div class="alert alert-warning">
    <b>Task</b>: What is the glaciological mass balance, and how is it measured? If you don't remember, take a look at the lecture slides!
</div>

We can look at this data for our glacier using a built-in OGGM function:

In [ ]:
mbdf_in_situ = gdir.get_ref_mb_data()
mbdf_in_situ

And plot the data:

In [ ]:
# plot the different components of the 
plt.plot(mbdf_in_situ['ANNUAL_BALANCE'], label='ANNUAL_BALANCE')
plt.plot(mbdf_in_situ['WINTER_BALANCE'], label='WINTER_BALANCE')
plt.plot(mbdf_in_situ['SUMMER_BALANCE'], label='SUMMER_BALANCE')

# make the plot nice with grid, title, legend and labels
plt.grid('on')
plt.title(f'{gdir.name}, {gdir.rgi_id}')
plt.legend()
plt.ylabel(r'specific mass-balance (mm w.e. yr$^{-1}$)')
plt.xlabel('Year');

<div class="alert alert-warning">
    <b>Task</b>: What is the difference between the individual terms? Can you see any trend? Also, compare it with the annual temperature shown above.
</div>

Your answer here:

## Monthly temperature index model

Now that we have defined the forcing climate data and looked at the available observations, we can move on to the actual mass balance model. You can find the equation and its description in the [OGGM documentation](https://docs.oggm.org/en/latest/mass-balance-monthly.html#monthly-temperature-index-model-calibrated-on-geodetic-mb-data).

<div class="alert alert-warning">
    <b>Task</b>: What do the different temperatures temp_melt, temp_all_solid, and temp_all_liq stand for?
</div>

Your answer here:

For the model, we need to calibrate two parameters that deal with downscaling the climate forcing data to the glacier location: the **precipitation factor** and the **temperature bias**. We also need to define the **melt factor** ($d_f$ in the equation).

This gives us three parameters, but we only have one observation, the **geodetic mass balance**, for each glacier. This is a problem, because it means the model is not well defined. There are infinitely many combinations of the three parameters that can match the single observation. If you're interested, you can find more details about this issue [this paragraph of the OGGM tutorials](https://tutorials.oggm.org/stable/notebooks/tutorials/massbalance_calibration.html#overparameteristion-or-the-magic-choice-of-the-best-calibration-option)).

In the next chapter, we willll see how OGGM deals with this. Keep in mind that this is just one possible approach. It's not the only way, or necessarily the best way. For example, other approaches are explained in [Huss and Hock 2015](https://doi.org/10.3389/feart.2015.00054) or [Rounce et al. 2020](https://doi.org/10.1017/jog.2019.91).

## Calibration of the mass balance model

As discussed before, we have three parameters we want to tune, but only one observation per glacier at the global scale. To handle this, we use methods that provide an educated first guess for two of the parameters:

### Method 1: Data informed precipitation factor

We developed a parameterisation for the precipitation factor, as described in [Schuster et al. 2023](https://doi.org/10.1017/aog.2023.57). You can read more about the method in [this paragarph of the OGGM tutorials](https://tutorials.oggm.org/stable/notebooks/tutorials/massbalance_calibration.html#first-step-data-informed-precipitation-factor).

In essence, they used data from around 100 glaciers that have additional direct mass balance observations (like the ones we looked at earlier). Based on this data, they created a parameterisation. Below, you can see a plot showing the final relationship between winter daily mean precipitation and the precipitation factor (plot taken from the OGGM tutorials).

In [ ]:
# define the range of winter precipitation to show in the plot
w_prcp_array = np.linspace(0.5, 20, 51)

# define the parametrisation and calculate the precipitation factor
a, b = cfg.PARAMS['winter_prcp_fac_ab']
r0, r1 = cfg.PARAMS['prcp_fac_min'], cfg.PARAMS['prcp_fac_max']
prcp_fac = a * np.log(w_prcp_array) + b
prcp_fac_array = utils.clip_array(prcp_fac, r0, r1)

# plot it
plt.plot(w_prcp_array, prcp_fac_array)
plt.xlabel(r'winter daily mean precipitation' +'\n'+r'(kg m$^{-2}$ day$^{-1}$)')
plt.ylabel('precipitation factor (prcp_fac)');

<div class="alert alert-warning">
    <b>Task</b>: Take a look at the plot: what does the relationship tell us? Why do you think winter precipitation is used instead of annual precipitation?
</div>

Your answer here:

### Method 2: Data informed temperature bias

We tried to identify general patterns in the temperature bias of the climate forcing data. You can read more about the methodology in [this paragarph of the OGGM tutorials](https://tutorials.oggm.org/stable/notebooks/tutorials/massbalance_calibration.html#second-step-data-informed-temperature-bias).

For this, we used the previously defined precipitation factor and a fixed melt factor value. Then, we calibrated the model by adjusting only the temperature bias to match the geodetic mass balance observation.

The plot below shows the resulting temperature bias globally, for each grid point of the climate data (plot and sentence below the plot are taken from the OGGM tutorials).

![err](https://user-images.githubusercontent.com/10050469/224318400-ec1d8825-d7e7-4cdb-94f3-ebb95b8f7120.jpg)

The fact that the `temp_bias` parameter is spatially correlated (many regions are all blue or red) indicate that something in the data needs to be corrected for our model.

<div class="alert alert-warning">
    <b>Task</b>: Take a look at the plot: can you identify which regions have a positive temperature bias and which have a negative one?
</div>

Your answere here:

### Final calibration of the mass balance model

With the two methods above, we now have a first guess for both the **precipitation factor** and the **temperature bias**. We can use these values to calibrate the **melt factor** using the **geodetic mass balance**. The exact calibration method is described in [this paragraph of the OGGM tutorials](https://tutorials.oggm.org/stable/notebooks/tutorials/massbalance_calibration.html#third-step-local-calibration-using-the-data-informed-precipitation-factor-and-temperature-bias-as-first-guesses).

Let’s calibrate it:

In [ ]:
# the default mb calibration
workflow.execute_entity_task(tasks.mb_calibration_from_geodetic_mb,
                             gdirs,
                             # informed_threestep means to use the two tricks described above, only avaiable for 'GSWP3_W5E5'
                             informed_threestep=True,
                            );

<div class="alert alert-warning">
    <b>Task</b>: Have a look in your glacier's working directory. Is there a new file?
</div>

You should now see a new file called `mb_calib.json`.

Let’s take a look at it:

In [ ]:
mb_calib = gdir.read_json('mb_calib')
mb_calib

<div class="alert alert-warning">
    <b>Task</b>: Do you understand what all the variables in this file mean?
</div>

Your answer here:

## Have a closer look at the resulting mass balance

Now that we have calibrated our mass balance model, we can look at the results and interact with it. You can access the model simply by using:

In [ ]:
from oggm.core import massbalance

mbmod = massbalance.MonthlyTIModel(gdir)

Internally, the mass balance model reads the `mb_calib.json` file and loads all the necessary parameters:

In [ ]:
mbmod.calib_params

### Mass balance profile

As a first step, we can look at the mass balance profile for two different years. To do this, we first extract the height and width values from the elevation band flowline we created in the last session:

In [ ]:
# extracting the heights and the widths of the inversion flowline
heights, widths = gdir.get_inversion_flowline_hw()

Now, let’s plot the annual mass balance profile along our elevation band flowline for two different years:

In [ ]:
# defining the years we want to plot, you also could try out different years
year_1 = 2000
year_2 = 2001

# plot the mass balance profiles of the two years
plt.plot(mbmod.get_annual_mb(heights, year=year_1), heights, label=year_1)
plt.plot(mbmod.get_annual_mb(heights, year=year_2), heights, label=year_2)

# add some labels, a legend and a grid
plt.ylabel('Elevation (m a.s.l.)'); plt.xlabel('MB (mm w.e. yr$^{-1}$)');
plt.legend(); plt.grid('on');

<div class="alert alert-warning">
    <b>Task</b>: Which year was more favorable for the glacier, and why?
</div>

Your answere here:

You can also get the equilibrium line altitude (ELA), for example, using the following command:

In [ ]:
mbmod.get_ela(year=2000)

<div class="alert alert-warning">
    <b>Task</b>: Do you remember from the lecture what the ELA represents?
</div>

Your answere here:

### Sensitivity of mass balance parameters

Let’s now explore how the mass balance model profiles respond to changes in its parameters. To do this, we will create a new file called `mb_calib_pertubed_melt_f.json`, where we change the melt factor to 7:

In [ ]:
# read the original paramters file
params = gdir.read_json('mb_calib')

# change the melt factor
params['melt_f'] = 7

# save the perturbed parameters as a new file with a filesuffix
gdir.write_json(params, 'mb_calib', filesuffix='_perturbed_melt_f')

And we can simply specify a filesuffix for the `mb_calib` file when defining the mass balance model:

In [ ]:
mbmod_perturbed_melt_f = massbalance.MonthlyTIModel(gdir, mb_params_filesuffix='_perturbed_melt_f')
mbmod_perturbed_melt_f.calib_params

Now we can take a look at how this change influenced our mass balance profile:

In [ ]:
year = 2000  # we want both mass balance models to use the same year

# the actual plotting
plt.plot(mbmod.get_annual_mb(heights, year=year), heights, label=f'{year} original')
plt.plot(mbmod_perturbed_melt_f.get_annual_mb(heights, year=year), heights, label=f'{year} perturbed melt_f')

# adding labels, a legend and a grid
plt.ylabel('Elevation (m a.s.l.)'); plt.xlabel('MB (mm w.e. yr$^{-1}$)');
plt.legend(); plt.grid('on');

<div class="alert alert-warning">
    <b>Task</b>: What could you observe? Where is the influence the strongest? Conduct the same sensitivity experiment for the precipitation factor and the temperature bias.
</div>

Your answer here:

### Compare to direct glaciological in-situ mass balance observations

The glaciological observations are provided as a single value per glacier and represent the specific mass balance. We can also easily obtain this from our mass balance model:

In [ ]:
mbmod.get_specific_mb(heights=heights, widths=widths, year=2000)

<div class="alert alert-warning">
    <b>Task</b>: How is the specific mass balance calculated and why do we need to provide heights AND widhts?
</div>

Your answer here:

With this, we can now plot the observation and the model output for comparison:

In [ ]:
# load the in-situ observations
mbdf_in_situ = gdir.get_ref_mb_data()

# for easier plotting we add a new column with our model data
mbdf_in_situ['modeled_mb'] = mbmod.get_specific_mb(heights=heights,
                                                   widths=widths,
                                                   # here we provide multiple years at once
                                                   year=mbdf_in_situ.index,  
                                                  )

# plot it
plt.plot(mbdf_in_situ['modeled_mb'], label='model output')
plt.plot(mbdf_in_situ['ANNUAL_BALANCE'], label='in-situ observations')

# add labels, legend and grid
plt.ylabel(r'specific mass-balance (mm w.e. yr$^{-1}$)'); plt.xlabel('Year');
plt.legend(); plt.grid('on');

Doesn't look too bad, but remember, the in-situ observations were used during the development of the precipitation factor parameterisation. Therefore, this is not an independent comparison.

<div class="alert alert-warning">
    <b>Bonus Task</b>: Add the perturbed mass balance results from the previous section to the plot, and interpret the differences. What effect does changing the mass balance parameters have on the model output compared to the original calibration and the observations?
</div>

Your answer here:

## Recap

- To force the OGGM mass balance model, we need temperature and precipitation data.
- There is only one geodetic mass balance observation available for calibration per glacier, globally.
- Some glaciers have additional in-situ observations, which can be used to develop parameterisations.
- The mass balance model is overparameterised: we have three parameters, but only one observation.
- We can apply some strategies to estimate two of the parameters, leaving one to calibrate directly.
- The three model parameters each have a different effect on the shape and values of the mass balance profile.